### 自动求导
对函数$y=2x^Tx$,关于列向量求导

In [71]:
import torch

In [72]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于x的梯度之前，我们需要一个地方来存储梯度（y关于x的导数）

In [73]:
x.requires_grad_(True)  # 等价于 x = torch.arange(4.0, requires_grad=True)
x.grad  # 默认值为None

In [74]:
# 计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

pytorch 隐式的构造计算图，

### 通过调用反向传播函数来自动计算y关于x的每个分量的梯度

In [75]:
y.backward() # 求导
x.grad

tensor([ 0.,  4.,  8., 12.])

求导$(2x^2)' = 4x$  
$\frac{\partial y}{\partial x} = \frac{\partial 2x^2}{\partial x} = 4x$

In [45]:
# 验证求导
x.grad == 4 * x

tensor([True, True, True, True])

In [48]:
# 默认情况下，pytorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum() # y = x1 + x2 + ... + xn = 0 + 1 + 2 + 3
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [51]:
# 对非标量调用‘backward需要传入一个gradient参数，该参数指定微分
x.grad.zero_()
y = x * x
# 等价于 y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 将某些计算移动到记录的计算图之外

In [52]:
x.grad.zero_()
y = x * x
u = y.detach() # u 不再是个函数，只是个常量,无法再进行求导
z = u * x  # u为常数，则 (ux)' = x 

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [54]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [56]:
# 即使构建函数的计算图需要用过python控制流（例如条件，循环），我们仍然可以计算得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000: # 2范数，即平方和开根
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a .grad == d / a

tensor(True)

In [63]:
x1 = torch.tensor(3.0, requires_grad=True)
y1 = torch.tensor(2.0, requires_grad=True)
a = x1 ** 2
b = 3 * a
c = b * y1
c.backward()
x1.grad

tensor(36.)

In [64]:
y1.grad

tensor(27.)

In [65]:
x1.grad == 6 * x1 * y1

tensor(True)

In [69]:
y1.grad == 3 * (x1 ** 2)

tensor(True)